<a href="https://colab.research.google.com/github/lyc760214/teaching_and_reporting/blob/master/s2s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 使用seq2seq模型進行翻譯 Translation with the seq2seq model



---







## 人工翻譯與機器翻譯

當人們面對不是母語的語言時，翻譯的步驟大致上可以分成：


1.   解析原始文句的文意
2.   重新編譯解析結果轉換至目標語言

在這看似簡單的步驟其實背後有著複雜的認知操作，一個譯者要必須能夠分析與詮釋整段文章的所有特徵，必須要暸解其文法、語意，甚至是慣用法、成語等，相當於要暸解來源語言的的文化背景。

當這樣的過程交由機器處理時，要設計程式讓機器「暸解」一段文字並能夠「創造」一段類似於人工翻譯的語句時，變成為一大挑戰。



## 類神經網路的「再」興起

類神經網路的概念於1980年代就已經提出，但由於遇到運算瓶頸的緣故很快的沒落下去，且在當時無法解決非線性資料的最佳化問題而使得研究進展遲滯不前。在當時，學界只要看到出現「神經網路」一詞的論文或研究計畫，便立刻被貶斥，認為多層的神經網路學習是不可能的，而造成僅有少數研究者繼續堅持於這個領域。

1990年代，另一類被稱為淺層機器學習開始大放異彩。其中支持向量機(support vector machines, SVM)最廣受歡迎。尤其當代，支持向量機在圖像和語音辨識上的成功，更使得類神經網路的研究陷入前所未有的低潮。

2006年，由於硬體運算效能的進步以及類神經網路在最佳化問題上的突破，多層的類神經網路終於重見天日。2012年的10月，由史丹佛大學舉辦的一場全世界最大圖像資料庫ImageNet的圖像辨識競賽中，由「深度學習 + GPU」的組合以16.42%的錯誤率大勝由「淺層學習」為主的第二名26.22%，這一戰讓類神經網路真正翻身。當然連帶的讓提供GPU運算技術的輝達(nVIDIA)更是一飛沖天。



---





## 類神經網路的種類

在此我們借用來自於 Asimov institute 的 Fjodor van Veen 所完成的類神經網路的種類圖：


![ANN-topology](https://drive.google.com/uc?export=view&id=1jnbdz-qlfajXd6m9yBqtmFSiG3NtrTmU)


在這之中，深度卷積神經網路 (Deep Convolutional Network, DCN) 已被大量用於圖像辨識上。另外，遞歸神經網路 (Recurrent Neural Network, RNN) 則經常用於自然語言的語意分析。


若有興趣了解各種網路的介紹可以參考：https://towardsdatascience.com/the-mostly-complete-chart-of-neural-networks-explained-3fb6f2367464

或是 TechOrange 的譯文：https://buzzorange.com/techorange/2018/01/24/neural-networks-compare/




---



## 長短期記憶神經網路 (Long/Short Term Memory)

長短期記憶神經網路是遞歸神經網路的一種延伸，因此我們先看一下遞歸神經網路的架構：

![替代文字](https://drive.google.com/uc?export=view&id=1LpKmicxiMnR4B9rLowsW1Mo_fbSegQ_C)

(圖片來源：http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

每一筆待預測的數據 X ，除了產生結果 h 以外，還額外傳遞了另一筆輸出作為下一筆待預測數據的額外輸入。藉由這種方式使得每一次的預測受到上一次預測結果的影響，而這樣的架構被稱為短期記憶。

做為「有記憶」的類神經網路架構，遞歸神經網路能夠處理與時間序有關的問題，也因此對於語言這一類前後內容彼此有關聯的問題，能夠有較佳的預測結果。

然而，由於記憶只受到「上一次結果」所影響，實際在面對問題時仍顯不夠。



---



長短期記憶神經網路的架構則是：

![替代文字](https://drive.google.com/uc?export=view&id=14dyqXedxpq2HEYQEnBJUYhX1QCiBBHft)

(圖片來源：http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

與遞歸神經網路相比，除了短期記憶被作為下一筆資料的輸入外，該記憶還被傳入另一條通道進行累計，而使得一筆預測除了受到輸入數據 X 和短期記憶以外，還多受到另一條持續累計的額外輸入。

這樣的架構相比標準遞歸神經網路而言，能夠受到更長久之前的預測結果影響。





---




## Sequence to Sequence 架構

一種基於長短期記憶神經網路的複雜架構 - Sequence to Sequence 被建立出來，其中該框架可用於機器翻譯、文本摘要、會話建構甚至是圖像字幕。該框架可以利用大量的不同語言的已知翻譯對照去建立翻譯模型，或收集大量文獻的本文以及其摘要、標題的數據去產生新文章的摘要和標題，或利用一般客服與用戶之間的對話資訊去創建自動聊天機器人。

Sequence to Sequence 的架構如下：

![替代文字](https://drive.google.com/uc?export=view&id=109DBgT0A61ku_MknmLCwUWn8IjgAqFxY)

(圖片來源：https://ai.googleblog.com/2015/11/computer-respond-to-this-email.html)

Seq2Seq 主要分成兩個區塊 - 編碼層(encoder)與解碼層(decoder)，利用逐步將字句送入編碼層並轉換成語意向量後，再將語意向量經過解碼層輸出字句。

比較值得一提的是「語意向量」的概念。由於語意向量本身不屬於任何一種語言，換言之任何語言的句子只要是對應到同一個「意思」，其語意向量的呈現便會是類似的。這意味著，若我們想要將某英文語句翻譯成中文、日文、德文等不同語言，由於語意向量是相同的，因此我們不再需要花大量的時間建立多個不同的翻譯模型，只要能建立出統一的語意向量結果，便能加快訓練的過程。




---




## 訓練集來源

我們的資料集來自於 www.manythings.org 一站，共有 20133 筆「英文 - 簡體中文」的對照語句，並利用 ConvertZZ 軟體將簡體中文轉換成繁體中文並且做部分的在地話修正。

以下是資料來源以及轉換工具的連結：

http://www.manythings.org/anki/

https://github.com/flier268/ConvertZZ

轉換後的訓練集則可存於下列連結：

https://drive.google.com/open?id=1kSild5T-Vx88I_1G9tYnftCa-tNWFnY0

為了使用 Google Colab 所提供的免費運算機器且連結到 Google Drive 的檔案，我們必須安裝 PyDrive 並以下列程式碼進行下載：

In [0]:
!pip install -U -q PyDrive

     |████████████████████████████████| 993kB 2.8MB/s 


In [0]:

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 驗證並創建PyDrive的用戶端
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# 藉由Google Drive的檔案ID下載資料並命名為cmn.txt
downloaded = drive.CreateFile({'id': '1kSild5T-Vx88I_1G9tYnftCa-tNWFnY0'})
downloaded.GetContentFile('cmn.txt')


downloaded = drive.CreateFile({'id': '1NYrotDvzGNAsPJvXax6f8E_wPMumGtu9'})
downloaded.GetContentFile('s2s_2500.h5')

## 訓練集整理

讀入剛剛所下載的資料，並利用numpy套件進行整理。

特別注意的是，由於原始資料是文字，但類神經網路的基礎架構仍然是數學的運算，因此必須分別將中文的文字與英文的字母做成字典，並使用索引值進行後續的操作：

In [0]:
import numpy as np

data_path = 'cmn.txt'

# 向量化原始資料
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
# 讀入資料，由於含中文資訊所以使用utf-8的編碼
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    
# 由於資料量過大，我們只能先以前5000句作為範例
'''
for line in lines:
'''
for line in lines[0:5000]:
    try:
        input_text, target_text = line.split('\t')
    except:
        print(line)
        continue
    # 使用'tab'作為target_text的起始字元，並使用'\n'作為結束字元
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

# 各別排序所建立的中文與英文的字元並建立成字典
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

# 檢查匯入的語句數，以及中文與英文字典的資訊
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of total samples:', len(input_texts))
print('Number of unique input tokens in English:', num_encoder_tokens)
print('Number of unique output tokens in Chinese:', num_decoder_tokens)
print('Max sequence length for inputs in English:', max_encoder_seq_length)
print('Max sequence length for outputs in Chinese:', max_decoder_seq_length)



在Seq2Seq2的模型中，由於標點與分隔符號都會被視為是獨立的字元，因此在檢查英文的獨特字元數時會有大於52個字元(26個小寫字母與26個大寫字母)。而中文的獨特字元更是複雜，達到了1685個。這也是一般面對翻譯問題時，與中文有關的翻譯效果總是較差。

然後，我們要將輸入與輸出的資料轉換成一個三維矩陣，其中第一個維度代表的這是第幾筆句子，第二個維度則是一筆語句中的文字位置，第三個維度則代表某一個文字它是哪個文字。

一般而言，三維矩陣的數值使用單精浮點數(float32)，但受限於Googol Colab僅提供約12G的記憶體，加上中文的字典實在是太大，所以在此暫時用半精浮點數處理：


In [0]:
# 建立用於儲存輸入和輸出資料的三維空矩陣
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float16')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float16')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float16')

# 將匯入的資料轉換成矩陣模式
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data 必須比 decoder_input_data 提前一步
        # 並且decoder_target_data 不需包含起始字元'tab'
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

截至目前為止，我們完成了我們訓練集的輸入(英文)與輸出(中文)矩陣。


 ## 訓練模型的建立

緊接著我們要建立我們的Seq2Seq模型，其中暫定使用128維的語意向量：

In [0]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

# 宣告語意向量的維度
latent_dim = 128

# 建立 encoder LSTM 隱藏層
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# 捨棄 output，只保留記憶狀態 h 及 c
encoder_states = [state_h, state_c]

# 建立 decoder LSTM 隱藏層
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# 我們設置 decoder 返回序列以及其內部狀態
# 但內部狀態僅用於推論而不直接用於訓練
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# 定義模型，將利用encoder_input_data和decoder_input_data轉換成decoder_target_data
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 將訓練模型包裝，並使用rmsprop作為最佳化算法
# 並由於文字屬於類別資料而使用categorical_crossentropy作為損失函數
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

以上為我們的模型建構。

其中比較特別的是我們在訓練的過程中同時使用了英文的語句與中文的語句作為輸入資料，這是因為我們要訓練的對象是「語意向量」。換言之，我們的訓練是要盡可能地將夾在編碼與解碼層的語意向量找到最佳的解：

(新版的keras移除了匯出圖片時的dpi可選參數，因此圖片的解析度較差)



In [0]:
from keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True, rankdir='TB')

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

matplotlib.rcParams['figure.dpi'] = 100
img = mpimg.imread('model.png')
plt.imshow(img)
plt.axis('off')
plt.show()


## 模型的訓練

我們開始進行模型的訓練。

在此有兩個參數batch_size和epochs需要設定。

一般而言，當資料量較大時，我們無法一次完成所有資料的訓練，因此可以藉由分割成多批次去進行，而batch_size便是指每一個批次裡面含有幾筆資料。

由於類神經網路的參數眾多，一次完整的訓練往往無法使類神經網路內的所有參數達到收斂，因此必須要重複性的送入資料去訓練模型，而epochs則是指總共需要重複幾次的訓練。

此外，在每一次完整資料的訓練過程中，keras會將根據validation_split將資料分成訓練組與驗證組。

最後，在訓練完成後可以將整個模型存出。

In [0]:
batch_size = 64  # 一個批次要送幾筆資料進去
epochs = 5  # 總共重複幾次完整資料的訓練

model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
model.save('s2s.h5')

在訓練的過程中，loss和val_loss是兩個我們關注的數值。Loss是指我們所送進去80%作為訓練集的資料，在一次完整的訓練後其誤差的量化結果，而val_loss則是剩餘20%作為驗證集的資料，在一次完整的訓練後其誤差多少。

一般而言，由於模型是對於訓練集進行擬合，因此loss會小於val_loss。當然最終目的是希望兩個數值都能越趨近於0越佳。

如同之前所述，這個訓練過程是在訓練當英文語句進入編碼層後所產生的「內部記憶狀態」如何連結到「語意向量」，然後「語意向量」如何連接到「解碼的輸出」而轉變成中文語句。

當這個訓練過程完成後，我們便能用訓練完的參數去建構真的翻譯推論模型。




## 推論模型的建立

在完成編碼->語意向量->解碼，這個過程的訓練後。我們用來進行翻譯的推論模型為了能真正地面對未知的問題，我們必須將訓練模型中的中文語句輸入資料移除。

原先的訓練模型為了獲得適當的語意向量參數，必須要仰賴英文與中文語句同時輸入，進而得到最佳的參數值。

當語意向量妥善化後，推論模型便是利用編碼層的長短期記憶神經網路將編碼映射到語意向量，再將語意向量映射到解碼層的長短期記憶神經網路而推論可能的翻譯結果。

In [0]:
# 定義編碼器的取樣模型並包裝
encoder_model = Model(encoder_inputs, encoder_states)

# 定義解碼器的輸入模型
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 定義解碼器的LSTM架構(建立將語意向量轉換成輸出的模型)
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)

# 定義解碼器的內部記憶來自於編碼器
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

# 將所有解碼器架構包裝的模型
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# 建立將解碼器輸出結果翻譯成可讀語句的字典
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())



一樣，我們可以看看我們用來進行翻譯的推論模型的結構：

In [0]:
plot_model(encoder_model, to_file='encoder_model.png', show_shapes=True, rankdir='TB')
matplotlib.rcParams['figure.dpi'] = 96
img = mpimg.imread('encoder_model.png')
plt.imshow(img)
plt.axis('off')
plt.show()

In [0]:
plot_model(decoder_model, to_file='decoder_model.png', show_shapes=True, rankdir='TB')
matplotlib.rcParams['figure.dpi'] = 130
img = mpimg.imread('decoder_model.png')
plt.imshow(img)
plt.axis('off')
plt.show()

最後，我們將編碼器與解碼器兩個模型合併，獲得一個具有翻譯能力的Seq2Seq模型：

In [0]:
def decode_sequence(input_seq):
    # 將輸入的語句轉為狀態向量
    states_value = encoder_model.predict(input_seq)

    # 生成長度為1的空目標序列
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # 將'tab'符號作為起始字元放入空目標序列
    target_seq[0, 0, target_token_index['\t']] = 1.

    # 建立批次循環採樣，為求簡單，先假設一個批次只有一組序列
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # 取出預測出來的字元，轉換成可讀的文字
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # 離開條件：當遇到換行符號'\n'或是超過可預測的語句長度
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # 更新目標序列
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # 更新狀態
        states_value = [h, c]

    return decoded_sentence

我們來測試目前的模型是否具有翻譯能力，以當初用來訓練的語句隨機抽十句為測試樣本：

In [0]:
import random
random.seed(15)

for seq_index in random.sample(range(5000), 10):
    # 取訓練集的一部份用於測試
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

我們可以看到128維的語意向量僅進行5次的訓練是完全不夠的。



---



## 長時間訓練的結果

為了呈現較佳的翻譯模型，我們嘗試了512維，並對所有語句進行2500次的訓練。

然而由於這樣的訓練較為費時，而Google Colab所提供的免費運算資源並無法負擔這樣的計算量。

因此我們使用了自己的機器進行訓練，規格如下：
CPU: Intel(R) Xeon(R) CPU E3-1230 v5,
Memory: 16 GB
GPU: GeForce GTX 1060 - 3G

再匯入之前，為了避免先前的資料影響，會進行所有變數的清空。

In [0]:
# 清空先前所宣告的所有變數
%reset -f

下載另一個經過長時間訓練的模型s2s_2500.h5

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 驗證並創建PyDrive的用戶端
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# 藉由Google Drive的檔案ID下載資料並命名為s2s_2500.h5
downloaded = drive.CreateFile({'id': '1NYrotDvzGNAsPJvXax6f8E_wPMumGtu9'})
downloaded.GetContentFile('s2s_2500.h5')



In [0]:
from keras.models import Model, load_model
from keras.layers import Input
import numpy as np
import random
random.seed(15)

latent_dim = 512
data_path = 'cmn.txt'

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines:
    try:
        input_text, target_text = line.split('\t')
    except:
        print(line)
        continue
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')

for i, input_text in enumerate(input_texts):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.

model = load_model('s2s_2500.h5')

encoder_inputs = model.input[0]   # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output   # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]   # input_2
decoder_state_input_h = Input(shape=(latent_dim,), name='input_3')
decoder_state_input_c = Input(shape=(latent_dim,), name='input_4')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
            
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence

for seq_index in random.sample(range(20133), 10):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

從結果來看，正確的翻譯增加了不少，不過無法理解的句子也非常的多。

雖然有些語句翻譯的錯誤程度不低，但可以發現大部分的句子仍然能成句，不會出現真的利用隨機字元拼湊的結果。

當然，我們沒有針對最佳的「語意向量」維度去進行討論，也尚未與「英法翻譯」、「英德翻譯」這一類語言架構較為類似的案例進行討論。因為一次完整的訓練便要花上不少時間，即使對於小資料進行參數最佳化，也不一定在對於大資料是最佳的。


---



## 後記

最開始我們想解決的問題是以蛋白質序列作為輸入，希望利用Seq2Seq的模型試著預測蛋白質的二級結構。但在實作之前，我們仍必須先暸解整個Seq2Seq的架構與運作概念。

此外，我們所建立的Seq2Seq模型依舊是非常簡單的。目前Seq2Seq模型的確廣泛地用於翻譯或是前面所提到的各種與時間序有關的預測模型，但其他擁有較佳精準度的模型可能會增添更多的隱藏層，甚至使用另一種「雙向長短期記憶神經網路」去建構。

最後，希望過去尚未接觸過keras套件的同學們，能夠藉由我們的介紹有機會先暸解一些套件的名稱或是使用。



---

